# ЧАСТЬ 0. Общие сведения

## Вероятности

Метод максимального правдоподобия. Правдоподобие - вероятность (для непрерывных величин берется плотность вероятности) пронаблюдать такую выборку при условии что она была порождена распределением с заданными параметрами. В ML почти всегда предполагается, что наблюдения независимы, иначе формулы становятся чудовищными.
$L(\theta|X,Y) = P(X, Y| \theta) = \prod_{n=1}^N P(x_i, y_i|\theta) \to max$

Гипотезы. Простейший случай: гипотеза $H_0$ как наиболее вероятная и $H_1$ как альтернатива (открытие). Статистика - некоторая функция от наблюдений. p-value - вероятность пронаблюдать такую же или более экстремальную статистику при условии, что нулевая гипотеза верна. Для двустороннего теста с асимметричным распределением $p = 2\min\{P(T \geq t \mid H_0), P(T \leq t \mid H_0)\}.$

## knn

Судим о точке по k ее ближайшим соседям. Если есть набор точек в $R^N$ и метрика/расстояние $d = d(x_i, x_j)$, можем посчитать расстояние от некоторой точки $x_m$ до всех других. Гиперпараметрами модели являются метрика d, число соседей k, веса (1/n, где n номер соседа например). 

## Классификация. Наивный байесовский классификатор

Пусть нам задан набор примеров $\{(x_i, y_i)\}$, $i=1,..,N$, $x_i = x_{ij}$, $j=1..D$ .

$x_i$ - фичи/признаки, $y_i$ - метка, объект принадлежит классу ${0, 1}$ (D фичей N примеров). Вспомним формулу Байеса из теории вероятностей.

$$P(A|B) = \frac{P(B|A)P(A)}{P(B)} \Rightarrow P(C_k| x_i) = \frac{P(x_i|C_k)P(C_k)}{P(x_i)}$$

Наивность: признаки - также независимы $P(x_i|C_k) = \prod_{j=1}^D P(x_{ij}|C_k)$.

Если у нас все признаки бинарные, то все сводится к частотам: 

1) $P(C_k)\approx N_k/N$, где $N_k$ - число примеров $k$-го класса, $N$ - число примеров всего.
    
2) $P(x_{ij}|C_k)$ - частота $N_{kj}/N_k$, где $N_{kj}$ - число единиц в j-м признаке у представителей класса k. (если у нас 2 примера $x_1 = (1, 0)$, $x_2 = (1, 1)$, $y_1=1$, $y_2=2$, то $N_{11} = 1$, $N_{12}=0$).

$C = argmax(P(x_i|C_k)P(C_k))$

## Метрики 

**Метрики Бинарной Классификации (предсказания {0, 1}).** 

Precision = TP/(TP+FP), Recall = TP/(TP+FN), F1 = 2PR/(P+R). Если открыть Википедию, там их будет примерно 1005000, надо выбирать исходя из смысла задачи.

ROC-AUC. ROC - График зависимости Recall от FP/(FP+TN). AUC - площадь под графиком.

**Метрики Регрессии (предсказания признака $y\in\mathbb R$).** Если $\widehat y_i$ - предсказания модели, то:

MSE: $\sum_{k=1}^N (y_i - \widehat y_i)^2$

MAE: $\sum_{k=1}^N |y_i - \widehat y_i|$

Доля объясненной дисперсии: $R^2 = 1 - \sum_{i=1}^N (y_i - \widehat y_i)/\sum (y_i - \mu)$, $\widehat y_i$ - прогнозы, $\mu$ - выборочное среднее. Это доля объясненной дисперсии :)

# ЧАСТЬ 1. Линейные модели

## Регрессия. Линейная регрессия

Формула $y_i = w\cdot x_i$, $x_i = (1, x_{i1}, x_{i2}, ... x_{iD})$, $w = (w_0, w_1,...,w_D)$, $i=1,..,N$ (D фичей + 1 фиктивная для сдвига, N примеров). Функция потерь (лосс) по методу наименьших квадратов $L = \sum_{i=1}^N (y_i - w\cdot x_i)^2\to min$.

Для регрессии нужно нормировать данные:

StandardScaler: ($z = (x - u) / s$). MinMaxScaler:  ($(x - x.min) / (x.max - x.min)$).

Аналитическое решение (нуль производной по w): конкатенируем $x_i, y_i$ в матрицы $X \in R^{N, D+1}$, $Y \in R^{N}$, в силу независимости все записывается в матричном виде. Итоговое решение $w = (X^TX)^{-1}X^TY$. Проверка по размерностям $dim X^TX = (D+1, N)\times(N, D+1), dim X^T y = (D+1, N)\times N$, получается размерность $D+1$, то есть правильная.

Численное решение. Задаем каким-нибудь образом  первый шаг алгоритма $w^{(0)}$ - первое приближение. Градиентный спуск на k-м шаге: обновляем веса в направлении антиградиента (уменьшаем функцию потерь)
$$\frac{\partial L}{\partial w_j}  =  \sum_{i = 1}^{N} (w^{(k-1)}\cdot x_i - y_i)x_{ij}$$
$$w^{(k)}_j := w^{(k-1)}_j - \alpha \frac{\partial L}{\partial w_j},$$
$\alpha$ — learning rate, гиперпараметр.

Четыре предположения линейной регрессии выведем из метода максимального правдоподобия:
    1. Наблюдения независимы;
    2. Зависимость y(x) действительно линейна;
    3. Ошибки распределены нормально $\mathscr {N}$ - функция распределения;
    4. Их дисперсии одинаковы;

$p(D|\theta) =(2)= p(y|X, w) =(1)= \prod_{i=1}^N p(y_i|x_i, w) =(3, 4)= \prod_{i=1}^N \mathscr {N}(y_i|wx_i, \sigma^2) = \prod_{i=1}^N\frac{e^{-(y_i-wx_i)^2/2\sigma^2}}{\sqrt{2\pi\sigma^2}}\to max$
Переходя к логарифмам, вынося $\sigma^2$ за скобки в силу (4), получаем
$\sum_{i=1}^D (y_i - wx_i)^2\to min$.

Инженерия Фич. Можем взять дополнительные фичи $x^2,x^3...$, $e^{-c(x-a)^2}$

Регуляризация: модифицированная функция потерь $L_p$ задается формулой
$\sum_{i=1}^N (y_i - wx_i)^2 + \alpha|w|^{p}$. У $L_2$ есть явная формула $w = (X^TX + \alpha I)^{-1}X^TY$. $L_1$ зануляет линейно зависимые веса.

**Аналитический вывод зануления весов для $L_1$ в случае двух линейно зависимых переменных $(x^{(1)}, x^{(2)})$:**

$$L = ||Xw - y||^2 + \lambda ||w||= \sum_{k = 1}^N ( x^{(1)}_k w_1 +  x^{(2)}_k w_2 + w_3- y)^2 + \lambda\sum_{j =1 }^2 |w_j|,$$

где 

1) признаки линейно зависимы $x^{(2)} = c x^{(1)}$, 

2) истинная зависимость имеет вид $y= a_1 x^{(1)} + a_2 x^{(2)} + b$.
    
Коэффициенты $a_1$ и $a_2$ при этом неоднозначны: если $y = Ax^{(1)}$ и $x^{(2)} = c x^{(1)}$, то $y = (A - Bc) x^{(1)} + Bx^{(2)}$ для произвольного $B$, это отчасти и дает возможность занулить один из весов. 

Для упрощения предположим еще что $b = w_3$.

Тогда 
$$L = \sum_{k = 1}^N ( x^{(1)}_k w_1 +  c x^{(1)}_k w_2 - (a_1 x^{(1)}_k + a_2 c x^{(1)}_k))^2 + \lambda ||w|| = ( w_1 +  c  w_2 - (a_1 + a_2 c ))^2 \sum_{k = 1}^N \left(x^{(1)}_k\right)^2 + \lambda ||w|| =$$
$$= ( w_1 +  c  w_2 - (a_1 + a_2 c ))^2 M + \lambda ||w||,$$
где $M = \sum_{k = 1}^N \left(x^{(1)}_k\right)^2$.


Возьмем частные производные

$$\frac{\partial L}{\partial w_1} = [2M ( w_1 +  c  w_2 - (a_1 + a_2 c ))] + \lambda{\rm sign}\, w_1$$
$$\frac{\partial L}{\partial w_2} = [2M ( w_1 +  c  w_2 - (a_1 + a_2 c ))]c + \lambda {\rm sign}\,  w_2$$

Для краткости
$$\frac{\partial L}{\partial w} = v_1 + v_2$$

В первом квадранте угол наклона вектора $v_2 = (\lambda {\rm sign}\, w_1,  \lambda {\rm sign}\,  w_2)^T$ равен $\pi/4$, а вектора  $v_1$ равен $(1, c)^T$, и при $c\ne 1$ -- не совпадает с ним. Применение градиентного спуска приводит к тому, что (поскольку выражение $( w_1 +  c  w_2 - (a_1 + a_2 c ))$ меняет знак) точка $(w_1, w_2)$ приближается сначала к ``оптимальной прямой'' - той самой линии уровня $L_1$, модули проекций $v_1$ и $v_2$ на ортогональное дополнение к которой приближенно равны а сами векторы разнонапрвлены. Затем,  осциллируя вокруг этой прямой, веса смещаются к одной из осей координат (определяемой значением $c$), и колеблются вокруг точки пересечения с осью уже до конца процесса.


## Классификация. SVM

Поиск гиперповерхности, разделяющей данные наилучшим образом.
$w\cdot x - b = 0$ делит пространство на две части. Отступы: $M_i = y_i (w\cdot x_i - b)$. Умножение на константу не меняет уравнение, поэтому можем нормировать отступы к единице.  В случае линейно разделимой выборки отступы при правильном выборе прямой положительны. Цель - максимизировать отступ до опорных векторов (на которых достигается минимум $M_\pm$, который можно считать равным 1 за счет нормировки). Формулы: $|w|^2\to min$, $M_i\ge 0$.

Линейно неразделимая выборка $0.5|w|^2 + C\sum_1^l \xi_i\to min$, $M_i\ge 1 - \xi_i$, $\xi_i\ge 0$.

Метод очень хорошо изучен и ему посвящено примерно 100500 работ.

## Классификация. Логистическая регрессия

Цель: превратить формулу регрессии в формулу для оценки вероятностей. Функция $\frac{1}{1 + e^{-z}}, z = w\cdot x$ переводит прямую в отрезок $[0, 1]$. Для вывода лосса воспользуемся формулой максимального правдоподобия $\prod_i p_i^{y_i}(1-p_i)^{(1-{y_i})}$.

$L(\sigma(z), y_i) = \prod_{i=1}^n \sigma(z)^{y_i}(1-\sigma(z))^{(1-y_i)}$

Классический трюк теории вероятностей: превратим произведение в суммму, логарифмируя. И еще возьмем минус, чтобы минимизировать функцию потерь
$L_{log}(\sigma(z), y_i) = -\sum_{i=1}^n [ y_i\ln\sigma(z) + (1 - y_i)\ln(1-\sigma(z))]$

$(L_{log}(\sigma(z), y_i))'_{w_j} = (\sigma(z) - y_i)x_j$

Обобщение на случай k классов: Softmax

Вероятность k-го из K классов $p_k(x_i) = \frac{e^{z_k(x_i)}}{\sum \limits_{j=1}^{K} e^{z_j (x_i)}}, z_k (x_i) = w_k \cdot x_i$.

Функция потерь и ее производная (в векторной форме) $L = - \sum \limits_{i=1}^{n} \sum \limits_{k=1}^{K} y_k^{(i)}
log(\hat p_k^{(i)})$, $\frac{\partial L}{\partial w_k} = \sum \limits_{i=1}^{n} (\hat p_k^{(i)} - y_k^{(i)}) x_i$

# ЧАСТЬ 2. Деревья

## Регрессия и классификация.

Интуитивно понятная структура, в каждой вершине сплит, идем вправо влево. Как правило, используется разбиение по одному признаку в одной вершине.

Из структуры дерева решений следует несколько интересных свойств:

    1) выученная функция — кусочно-постоянная, из-за чего производная равна нулю везде, где задана. Следовательно, о градиентных методах при поиске оптимального решения можно забыть;
    2) дерево решений (в отличие от, например, линейной модели) не сможет экстраполировать зависимости за границы области значений обучающей выборки;
    3) дерево решений способно идеально приблизить обучающую выборку и ничего не выучить (то есть такой классификатор будет обладать низкой обобщающей способностью): для этого достаточно построить такое дерево, в каждый лист которого будет попадать только один объект. 

Построение дерева жадным алгоритмом: вводим критерий ветвления (функция, измеряющая, насколько хорош предлагаемый сплит) $H(X_m)−\frac{∣X_l∣}{∣X_m|}H(X_l)−\frac{∣X_r∣}{∣X_m∣}H(X_r)$, где $H$ - некоторая функция информативности, $X_k$ - множество объектов, попавших в $k$-ю вершину.

Информативность MSE в регрессии
$H(X_m) = \frac{1}{|X_m|}min_c\sum_{(x_i, y_i)\in X_m} (y_i - c)^2$

Информативность в классификации. Доля объекта класса k в текущей вершине $p_k = N_k/N$.

Энтропия $H(X_m) = -\sum_{k=1}^K p_k\ln p_k$

Джини $H(X_m) = \sum_{k=1}^K p_k(1 - p_k) = 1 - \sum_{k=1}^K p_k^2$, так как $\sum p_k = 1$.

## Ансамбли

Беггинг. Пусть обучающая выборка состояла из n объектов. Выберем из неё n примеров равновероятно, с возвращением. Получим новую выборку $X_1$, в которой некоторых элементов исходной выборки не будет, а какие-то могут войти несколько раз. С помощью некоторого алгоритма b обучим на этой выборке модель $b_1(x)=b(x,X_1)$. Повторим процедуру: сформируем вторую выборку $X_2$ из n элементов с возвращением и с помощью того же алгоритма обучим на ней модель $b_2(x)=b(x,X_2)$. Повторив процедуру k раз, получим k моделей, обученных на k выборках. Чтобы получить одно предсказание, усредним предсказания всех моделей. Процесс генерации подвыборок с помощью семплирования с возвращением называется бутстрепом (bootstrap), а модели $b_1(x),…,b_k(x)$ часто называют базовыми алгоритмами/моделями. Модель a(x) называется ансамблем этих моделей.

Случайный лес. Построим ансамбль алгоритмов, где базовый алгоритм — это решающее дерево. Будем строить по следующей схеме: Для построения i-го дерева: Сначала, как в обычном бэггинге, из обучающей выборки X выбирается с возвращением случайная подвыборка $X_i$ того же размера, что и X. В процессе обучения каждого дерева в каждой вершине случайно выбираются $n<N_n<N$ признаков, где N — полное число признаков (метод случайных подпространств), и среди них ищется оптимальный сплит. Такой приём как раз позволяет управлять степенью скоррелированности базовых алгоритмов.

Бустинг.

Рассмотрим пример с задачей регрессии и квадратичной функцией потерь:
$L(y,x)=∑_{i=1}^N(y_i−a(x_i))^2\to min⁡$. Для решения будем строить композицию из K базовых алгоритмов семейства B: $a(x)=a_K(x)=b_1(x)+b_2(x)+⋯+b_K(x)$.

В качестве базовых алгоритмов выберем семейство $B$ решающих деревьев некоторой фиксированной глубины. Используя известные нам методы построения решающих деревьев, обучим алгоритм $b_1(x)∈B$, который наилучшим образом приближает целевую переменную: $b_1(x)=argmin_{b∈B} L(y,b(x))$.

Построенный алгоритм $b_1(x)$, скорее всего, работает не идеально. Более того, если базовый алгоритм работает слишком хорошо на обучающей выборке, то высока вероятность переобучения. Вычислим, насколько сильно отличаются предсказания этого дерева от истинных значений: $s^{(1)}_i=y_i−b_1(x_i)$

Теперь мы хотим скорректировать $b_1(x)$ с помощью $b_2(x)$:
$b_2(x)=argmin_{b∈B} L(s^{(1)},b(x))$

Далее рассуждения повторяются до построения всей композиции. На k-ом шаге вычисляется разность между правильным ответом и текущим предсказанием композиции из k−1 алгоритмов:
$s_i^{(k−1)}=y_i−\sum_{j=1}^{k−1}b_j(x_i)=y_i−a_{k−1}(x_i)$

Затем k-й алгоритм учится предсказывать эту разность:
$b_k(x)=argmin_{b∈B} L(s^{(k−1)},b(x))$,
а композиция в целом обновляется по формуле
$a_k(x)=a_{k−1}(x)+b_k(x)$


Отметим теперь важное свойство функции потерь в рассмотренном выше примере с регрессией. Посчитаем производную функции потерь по предсказанию $z=a_k(x_i)$ модели для i-го объекта:
$\frac{∂L(y_i,z)}{∂z}=2(a_k(x_i)−y_i)$

Видим, что разность, на которую обучается k-й алгоритм, выражается через производную.

Таким образом, для каждого объекта $x_i$ очередной алгоритм в бустинге обучается предсказывать антиградиент функции потерь по предсказанию модели $−∂L(y_i,z)/∂z$ в точке $a_k(x_i)$ предсказания текущей части композиции на объекте $x_i$.

Почему же это важно? Дело в том, что это наблюдение позволяет обобщить подход построения бустинга на произвольную дифференцируемую функцию потерь. 

Также для борьбы с переобучением можно умножать каждый следующий алгоритм на коэффициент learning rate: $a(x)=a_K(x)=b_1(x)+\alpha b_2(x)+⋯+\alpha b_K(x)$..